## Las Cortes

Scraping: Wikipedia

In [ ]:
# SCRAPEA Y GUARDA EL HTML DE RESULTADOS DE CADA ELECCION CON SOLO INFO DE LAS CORTES
# ASSET PARA DESPUES SACAR LA INFO EN JSON

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import os

# 📌 Configurar Selenium en modo headless (sin interfaz gráfica)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 📌 Iniciar WebDriver automáticamente con WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 📌 Diccionario con los años y URLs de las elecciones a las Cortes Valencianas en Wikipedia
election_urls = {
    "1983": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_1983",
    "1987": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_1987",
    "1991": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_1991",
    "1995": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_1995",
    "1999": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_1999",
    "2003": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_2003",
    "2007": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_2007",
    "2011": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_2011",
    "2015": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_2015",
    "2019": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_2019",
    "2023": "https://es.wikipedia.org/wiki/Elecciones_a_las_Cortes_Valencianas_de_2023"
}

# 📌 Carpeta donde se guardarán los archivos HTML
output_folder = "elecciones_html/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 📌 Recorrer todas las URLs y guardar el HTML completo
for year, url in election_urls.items():
    print(f"🔍 Guardando HTML de {year}...")

    driver.get(url)
    time.sleep(4)  # Esperar para que cargue la página

    # Obtener el HTML completo de la página
    page_html = driver.page_source

    # Guardar el HTML en un archivo
    html_path = os.path.join(output_folder, f"elecciones_{year}.html")
    with open(html_path, "w", encoding="utf-8") as file:
        file.write(page_html)

    print(f"✅ Guardado: {html_path}")

# 📌 Cerrar Selenium
driver.quit()
print("\n🚀 Descarga completada. Archivos guardados en la carpeta 'elecciones_html/'")

In [ ]:
# LEE LOS ARCHIVOS HTML LIMPIOS CON SOLO LA TABLA Y 
# GENERA UN JSON PARA LOS DIPUTADOS DE CADA LEGISLATURA
# EN UN NUEVO FOLDER ./resultados_json

import os
import json
from bs4 import BeautifulSoup

# Directorio donde están los archivos HTML
folder_path = "./"
output_folder = os.path.join(folder_path, "resultados_json")

# Crear el folder de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Obtener la lista de archivos HTML en la carpeta
html_files = [f for f in os.listdir(folder_path) if f.endswith(".html")]

# Iterar sobre cada archivo HTML
for html_file in html_files:
    file_path = os.path.join(folder_path, html_file)

    # Leer el archivo HTML
    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    # Encontrar todas las filas de la tabla
    tbody = soup.find("tbody")
    if not tbody:
        print(f"No se encontró tbody en {html_file}. Saltando...")
        continue
    
    rows = tbody.find_all("tr")

    diputados = []
    current_party = None

    # Iterar sobre las filas para extraer los datos
    for row in rows:
        cols = row.find_all("td")
        
        if len(cols) >= 2:
            if cols[1].b:  # Identificar el partido
                current_party = cols[1].get_text(strip=True).split("\n")[0]
                num_diputados = int(cols[1].get_text(strip=True).split("(")[-1].split()[0])
            else:
                num_diputados = None
            
            # Obtener nombres de diputados por circunscripción
            circunscripciones = ["Alicante", "Castellón", "Valencia"]
            diputados_por_circunscripcion = {}

            for i, circ in enumerate(circunscripciones):
                if len(cols) > i + 2:
                    diputados_lista = [li.get_text(strip=True) for li in cols[i + 2].find_all("li")]
                    diputados_por_circunscripcion[circ] = diputados_lista

            # Guardar los datos
            diputados.append({
                "partido": current_party,
                "total_diputados": num_diputados,
                "diputados_por_circunscripcion": diputados_por_circunscripcion
            })

    # Definir el nombre del archivo JSON dentro de la carpeta de resultados
    json_file_name = html_file.replace(".html", ".json")
    json_file_path = os.path.join(output_folder, json_file_name)

    # Guardar en JSON dentro del folder
    with open(json_file_path, "w", encoding="utf-8") as json_file:
        json.dump(diputados, json_file, ensure_ascii=False, indent=4)

    print(f"Archivo JSON generado con éxito: {json_file_path}")


In [ ]:
# LEE LOS JSON Y UNIFICA EN UN SOLO CSV

import os
import json
import csv

# Diccionario con las legislaturas y sus rangos de años
legislaturas = {
    "I": (1983, 1987),
    "II": (1987, 1991),
    "III": (1991, 1995),
    "IV": (1995, 1999),
    "V": (1999, 2003),
    "VI": (2003, 2007),
    "VII": (2007, 2011),
    "VIII": (2011, 2015),
    "IX": (2015, 2019),
    "X": (2019, 2023),
    "XI": (2023, 2025)  # 2025 como "actualidad" temporal
}

# Directorio donde están los archivos JSON
folder_path = "./resultados_json"
output_file = "diputados_consolidado.csv"

# Lista para almacenar los datos
datos = []

# Obtener la lista de archivos JSON en la carpeta
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

# Iterar sobre cada archivo JSON
for json_file in json_files:
    # Extraer el año desde el nombre del archivo (asumiendo formato 'diputados_AAAA.json')
    year = int(json_file.split("_")[1].split(".")[0])

    # Determinar la legislatura correspondiente
    legislatura_asignada = None
    for legislatura, (inicio, fin) in legislaturas.items():
        if inicio <= year < fin:
            legislatura_asignada = legislatura
            break

    # Leer el archivo JSON
    with open(os.path.join(folder_path, json_file), "r", encoding="utf-8") as file:
        diputados_data = json.load(file)

    # Extraer datos y agregarlos a la lista
    for partido in diputados_data:
        for provincia, diputados in partido["diputados_por_circunscripcion"].items():
            for diputado in diputados:
                datos.append([diputado, partido["partido"], provincia, legislatura_asignada])

# Guardar los datos en un CSV
with open(output_file, "w", encoding="utf-8", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=";")
    writer.writerow(["diputado", "partido", "provincia", "n_legislatura"])  # Cabecera
    writer.writerows(datos)

print(f"Archivo CSV generado con éxito: {output_file}")


## Consellers (Wayback Machine)

Scraping: Wayback Machine

In [ ]:
# Devuelve un JSON con la info de consellers hasta la Legislatura IX


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

# URL de la versión archivada
URL = "https://web.archive.org/web/20150923221316/http://www.argos.gva.es/va/institucional/consell/"

# Configurar Selenium con ChromeDriver Manager
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Ejecutar sin interfaz gráfica

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

driver.get(URL)

# Esperar que la página cargue completamente
time.sleep(5)

# Intentar encontrar los botones de legislaturas
try:
    wait = WebDriverWait(driver, 15)
    legislaturas = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.tab-menu li a")))

    data = {}

    for legislatura in legislaturas:
        nombre_legislatura = legislatura.text.strip()
        print(f"Extrayendo datos de {nombre_legislatura}...")

        # Usar JavaScript para hacer clic en la legislatura
        driver.execute_script("arguments[0].click();", legislatura)
        time.sleep(3)

        # Buscar las secciones de cada equipo de gobierno
        equipos_gobierno = driver.find_elements(By.CSS_SELECTOR, "table.tabla_demoscopia")
        data[nombre_legislatura] = []

        for equipo in equipos_gobierno:
            try:
                titulo_equipo = equipo.find_element(By.XPATH, "./tbody/tr[1]/td").text.strip()
                consejeros = equipo.find_elements(By.CSS_SELECTOR, "td ul li")
                lista_consejeros = [c.text.strip() for c in consejeros if c.text.strip()]
                
                if titulo_equipo and lista_consejeros:
                    data[nombre_legislatura].append({
                        "equipo": titulo_equipo,
                        "consejeros": lista_consejeros
                    })
            except Exception as e:
                print(f"Error extrayendo equipo de gobierno en {nombre_legislatura}: {e}")

    # Eliminar legislaturas vacías
    data = {k: v for k, v in data.items() if v}

    # Guardar los datos en un archivo JSON
    with open("consejeros_valencia.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    print("Extracción completada. Datos guardados en 'consejeros_valencia.json'.")

except Exception as e:
    print("No se encontraron los elementos esperados. Error:", e)
    print("HTML de la página para depuración:\n", driver.page_source)

driver.quit()

## Consellers (pag. Oficial Govern)

Scraping: Govern

In [ ]:
# POR FIN ITERAÇAO BUENA - RECOGE DATOS DE LOS CONSEJEROS DE LA PAG DEL GOVERN


import requests
from bs4 import BeautifulSoup
import json
import re
import time
import os
from datetime import datetime

def get_gobierno_info(url, nombre_legislatura):
    """
    Extrae la información de los gobiernos de una legislatura específica
    
    Args:
        url (str): URL de la página de la legislatura
        nombre_legislatura (str): Nombre o identificador de la legislatura
    
    Returns:
        dict: Información de la legislatura y sus gobiernos
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Realizar la solicitud HTTP
    print(f"Obteniendo información de: {nombre_legislatura} - {url}")
    response = requests.get(url, headers=headers)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code != 200:
        return {"error": f"Error al acceder a la página {nombre_legislatura}: {response.status_code}"}
    
    # Parsear el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Inicializar la estructura para almacenar la información
    legislatura = {
        "nombre": nombre_legislatura,
        "url": url,
        "gobiernos": []
    }
    
    # Buscar todos los gobiernos
    gobiernos_divs = soup.find_all('div', class_='governs')
    
    if not gobiernos_divs:
        return {"nombre": nombre_legislatura, "url": url, "error": "No se encontraron gobiernos en la página"}
    
    # Iterar sobre cada gobierno
    for gobierno_div in gobiernos_divs:
        gobierno_info = {}
        
        # Extraer el título/período del gobierno
        titulo_gobierno = gobierno_div.find('h2', class_='asset-title')
        if titulo_gobierno and titulo_gobierno.a:
            # Eliminar el onClick y extraer solo el texto
            gobierno_info["titulo"] = titulo_gobierno.a.text.strip()
            
            # Intentar extraer fechas del título usando regex
            fecha_match = re.search(r'(\d{2}\.\d{2}\.\d{4})-(\d{2}\.\d{2}\.\d{4})', gobierno_info["titulo"])
            if fecha_match:
                gobierno_info["fecha_inicio"] = fecha_match.group(1)
                gobierno_info["fecha_fin"] = fecha_match.group(2)
        
        # Encontrar el contenedor de información del gobierno
        gobierno_container = gobierno_div.find('div', class_='governsGva')
        if not gobierno_container:
            continue
        
        # Extraer información del presidente
        presidente_div = gobierno_container.find('div', class_='presidente')
        if presidente_div:
            presidente_info = {}
            
            # Extraer nombre del presidente
            nombre_span = presidente_div.find('span', class_='nombre')
            if nombre_span:
                presidente_info["nombre"] = nombre_span.text.strip()
            
            # Extraer cargo del presidente
            cargo_p = presidente_div.find('p', class_='cargo')
            if cargo_p:
                presidente_info["cargo"] = cargo_p.text.strip()
            
            gobierno_info["presidente"] = presidente_info
        
        # Extraer información de los consejeros
        consejeros = []
        consellers_divs = gobierno_container.find_all('div', class_='conseller')
        
        for conseller_div in consellers_divs:
            consejero_info = {}
            
            # Extraer nombre del consejero
            nombre_span = conseller_div.find('span', class_='nombre')
            if nombre_span:
                consejero_info["nombre"] = nombre_span.text.strip()
            
            # Extraer cargo del consejero
            cargo_p = conseller_div.find('p', class_='cargo')
            if cargo_p:
                # El cargo puede contener un enlace o ser texto plano
                cargo_link = cargo_p.find('a')
                if cargo_link:
                    consejero_info["cargo"] = cargo_link.text.strip()
                    consejero_info["enlace_nombramiento"] = cargo_link['href']
                else:
                    consejero_info["cargo"] = cargo_p.text.strip()
            
            # Añadir el consejero a la lista
            if consejero_info:
                consejeros.append(consejero_info)
        
        gobierno_info["consejeros"] = consejeros
        
        # Añadir el gobierno a la lista de gobiernos
        legislatura["gobiernos"].append(gobierno_info)
    
    return legislatura

def main():
    # Lista de URLs de las legislaturas
    legislaturas = [
        {"url": "https://presidencia.gva.es/es/govern-preautonomic", "nombre": "Gobierno Preautonómico"},
        {"url": "https://presidencia.gva.es/es/govern-de-transicio", "nombre": "Gobierno de Transición"},
        {"url": "https://presidencia.gva.es/es/i-legislatura", "nombre": "I Legislatura"},
        {"url": "https://presidencia.gva.es/es/ii-legislatura", "nombre": "II Legislatura"},
        {"url": "https://presidencia.gva.es/es/iii-legislatura", "nombre": "III Legislatura"},
        {"url": "https://presidencia.gva.es/es/iv-legislatura", "nombre": "IV Legislatura"},
        {"url": "https://presidencia.gva.es/es/v-legislatura", "nombre": "V Legislatura"},
        {"url": "https://presidencia.gva.es/es/vi-legislatura", "nombre": "VI Legislatura"},
        {"url": "https://presidencia.gva.es/es/vii-legislatura", "nombre": "VII Legislatura"},
        {"url": "https://presidencia.gva.es/es/viii-legislatura", "nombre": "VIII Legislatura"},
        {"url": "https://presidencia.gva.es/es/ix-legislatura", "nombre": "IX Legislatura"},
        {"url": "https://presidencia.gva.es/es/x-legislatura", "nombre": "X Legislatura"},
        {"url": "https://presidencia.gva.es/es/xi-legislatura", "nombre": "XI Legislatura"}
    ]
    
    # Crear un directorio con timestamp para almacenar los resultados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = f"gobiernos_gva_{timestamp}"
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Creado directorio para resultados: {output_dir}")
    
    # Inicializar la estructura para almacenar toda la información
    all_legislaturas = []
    
    # Iterar sobre cada legislatura
    for legislatura in legislaturas:
        # Obtener la información de la legislatura
        info = get_gobierno_info(legislatura["url"], legislatura["nombre"])
        
        # Guardar la información en un archivo JSON individual
        filename = f"{output_dir}/{legislatura['nombre'].replace(' ', '_').lower()}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(info, f, ensure_ascii=False, indent=4)
        
        print(f"Información guardada en: {filename}")
        
        # Añadir la información a la lista completa
        all_legislaturas.append(info)
        
        # Esperar un poco entre solicitudes para no sobrecargar el servidor
        time.sleep(1)
    
    # Guardar toda la información en un único archivo JSON
    with open(f"{output_dir}/todos_los_gobiernos_gva.json", 'w', encoding='utf-8') as f:
        json.dump(all_legislaturas, f, ensure_ascii=False, indent=4)
    
    # Guardar un resumen estadístico
    stats = {
        "fecha_ejecucion": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "total_legislaturas": len(legislaturas),
        "legislaturas_procesadas": len(all_legislaturas),
        "total_gobiernos": sum(len(leg.get("gobiernos", [])) for leg in all_legislaturas),
        "errores": [leg["nombre"] for leg in all_legislaturas if "error" in leg]
    }
    
    with open(f"{output_dir}/resumen.json", 'w', encoding='utf-8') as f:
        json.dump(stats, f, ensure_ascii=False, indent=4)
    
    print("\nProceso completado.")
    print(f"Toda la información ha sido guardada en el directorio: {output_dir}")
    print(f"Archivos generados:")
    print(f"- {len(legislaturas)} archivos individuales por legislatura")
    print(f"- Un archivo completo con todas las legislaturas: todos_los_gobiernos_gva.json")
    print(f"- Un archivo de resumen: resumen.json")
    
    # Mostrar las estadísticas
    print("\nResumen:")
    print(f"- Total de legislaturas procesadas: {stats['legislaturas_procesadas']}")
    print(f"- Total de gobiernos extraídos: {stats['total_gobiernos']}")
    if stats['errores']:
        print(f"- Legislaturas con errores: {', '.join(stats['errores'])}")
    else:
        print("- No se encontraron errores durante el proceso")

if __name__ == "__main__":
    main()

In [ ]:
# PROCESA JSON DE ENTRADA (CONSELL) Y TRANSFORMA A CSV DE SALIDA

import json
import csv

# Nombre de los archivos
json_file = "todos_los_gobiernos_gva.json"
csv_file = "consells_consolidado.csv"

# Cargar los datos del JSON
with open(json_file, "r", encoding="utf-8") as file:
    data = json.load(file)

# Lista para almacenar los datos extraídos
rows = []

# Iterar sobre cada legislatura
for legislatura in data:
    n_legislatura = legislatura["nombre"]  # Nombre de la legislatura

    # Iterar sobre cada equipo de gobierno dentro de la legislatura
    for gov in legislatura["gobiernos"]:
        n_govern = gov["titulo"]  # Título del gobierno

        # Incluir al presidente en la lista
        presidente = gov["presidente"]
        rows.append([n_legislatura, n_govern, presidente["nombre"], presidente["cargo"]])

        # Iterar sobre cada consejero
        for consejero in gov["consejeros"]:
            rows.append([n_legislatura, n_govern, consejero["nombre"], consejero["cargo"]])

# Guardar en CSV
with open(csv_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, delimiter=";")
    writer.writerow(["n_legislatura", "n_govern", "persona", "cargo"])  # Encabezados
    writer.writerows(rows)

print(f"✅ Archivo CSV generado: {csv_file}")